# Start a PBS cluster on `casper`

[documentation](https://github.com/NCAR/ncar-jobqueue)

In [2]:
!conda list distr

# packages in environment at /glade/u/home/aspring/.conda/envs/s2s-climpred:
#
# Name                    Version                   Build  Channel
distributed               2.30.1           py38h578d9bd_0    conda-forge


In [3]:
!conda list dask

# packages in environment at /glade/u/home/aspring/.conda/envs/s2s-climpred:
#
# Name                    Version                   Build  Channel
dask                      2.30.0                     py_0    conda-forge
dask-core                 2.30.0                     py_0    conda-forge
dask-jobqueue             0.7.2              pyhd8ed1ab_1    conda-forge


In [4]:
from ncar_jobqueue import NCARCluster
from dask.distributed import Client
cluster = NCARCluster(project='NASP0002')
cluster

/glade/u/home/aspring/.conda/envs/s2s-climpred/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38116 instead
  warnings.warn(


In [5]:
#import dask
#dask.config.set({'distributed.dashboard.link':'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'})

In [11]:
# choose either one or
cluster.scale(jobs=2) # no initial hickups
#cluster.adapt(minimum_jobs=1,maximum_jobs=8) # more economical

In [7]:
client = Client(cluster)

In [8]:
client

Client Scheduler: tcp://10.12.206.63:36876 Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/aspring/proxy/38116/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [9]:
print(cluster.job_script())

#!/usr/bin/env bash

#PBS -N dask-worker-casper-dav
#PBS -q casper
#PBS -A NASP0002
#PBS -l select=1:ncpus=1:mem=25GB
#PBS -l walltime=01:00:00
#PBS -e /glade/scratch/aspring/dask/casper-dav/logs/
#PBS -o /glade/scratch/aspring/dask/casper-dav/logs/

/glade/u/home/aspring/.conda/envs/s2s-climpred/bin/python -m distributed.cli.dask_worker tcp://10.12.206.63:36876 --nthreads 2 --memory-limit 25.00GB --name dummy-name --nanny --death-timeout 60 --local-directory /glade/scratch/aspring/dask/casper-dav/local-dir --interface ib0 --protocol tcp://



In [10]:
cluster.dashboard_link

'https://jupyterhub.hpc.ucar.edu/stable/user/aspring/proxy/38116/status'

In [ ]:
# at the end of the sessions

In [11]:
cluster.close()

/glade/u/home/aspring/miniconda3/envs/s2s/lib/python3.8/site-packages/dask_jobqueue/core.py:360: FutureWarning: ignoring was deprecated in version 2021.06.1 and will be removed in a future release. Please use contextlib.suppress from the standard library instead.
  with ignoring(RuntimeError):  # deleting job when job already gone
/glade/u/home/aspring/miniconda3/envs/s2s/lib/python3.8/site-packages/dask_jobqueue/core.py:360: FutureWarning: ignoring was deprecated in version 2021.06.1 and will be removed in a future release. Please use contextlib.suppress from the standard library instead.
  with ignoring(RuntimeError):  # deleting job when job already gone


In [12]:
client.close()